<a href="https://colab.research.google.com/github/EverardoG/ml_comprobofinal/blob/net/starter_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
"""
Handle imports in one standard location

Some of these may not be necessary
I copied them over from a different ml project
"""
import cv2                           # Image handling
import io                            # Handling zip object byte files
import json
import os
import time                          # Timer for image loading
import torch                         # Main ML Library
import zipfile                       # Zip file nterfacing
%matplotlib inline

import matplotlib.pyplot as plt      # Plotting & Visualization
import numpy as np                   # Matrix handling
import pandas as pd                  # Data storage & frameworks

from google.colab import files
from PIL import Image                # Image handling
from sklearn.model_selection import train_test_split
from torch import nn
from torch.autograd import Variable  # Data class for nn training and testing
from torch.utils.data.sampler import SubsetRandomSampler # Class for creating random train/test datasets
import pickle # for saving arbitrary objects
from torch.utils.data import Dataset
import torchvision
import torch.optim as optim

# Define Neural Network

In [ ]:
class StarterNN(nn.Module):
  def __init__(self):
    # Not sure what this does - It's from the pytorch docs
    super(StarterNN, self).__init__()

    # Set a verbosity for logging
    self.verbose = 0
  
    # # Defining an activation function
    # self.activation_func = torch.nn.ReLU()

    # We only have 3 features (x, y, v) : Position and velocity of the ball
    # and 1 output: v_N : velocity of the Neato
    # TODO: Add bias term that's just a 1 (can add that into linear layer instead of encoding as input... maybe)
    input_size = 3
    fc1_size = 1

    # fc1 stands for Fully Connected Layer 1
    self.fc1 = nn.Linear(input_size, fc1_size)
      
  def forward(self, x):
    # Run training data through first fc layer
    x = self.fc1(x)

    # Put output through our activation function to add non-linearity
    # x = self.activation_func(x)
    return x

  def getLossFunctionAndOptimizer(self, learning_rate):
    # Loss function
    loss = nn.MSELoss()
    # TODO: Use squared loss for this. Figure out what this is in. Don't use cross entropy

    # Optimizer, self.parameters() returns all the Pytorch operations that are attributes of the class
    optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
    # optimizer is probably fine

    return loss, optimizer

starter_net = StarterNN()
print(starter_net)

StarterNN(
  (fc1): Linear(in_features=3, out_features=1, bias=True)
)


# Define Simulation For Generating Fake Data

In [ ]:
"""
Columns for data are as follows: x, y, v_B, v_N
"""

def runSimulation(x, y, v_B, v_N, dt = 0.1, max_steps = 100, sim_verbosity = 1):
    """
    Inputs:
      x: Initial x position of the ball relative to the robot
      y: Initial y position of the ball relative to the robot
      v_B: Initial velocity of the ball
      v_N: Initial velocity of the Neato
      max_steps: Maximum number of steps to run simulation
      sim_verbosity: How verbose to be in printing

    Returns:
      data: numpy array of data collected from simulated run
    """

    # Run simulation for at a maximum of max_steps
    for num_step in range(max_steps):
        # Step x and y forward
        x = x - v_N * dt
        y = y - v_B * dt

        # Stop the run if the ball has passed the robot
        if y < 0:
          if sim_verbosity >= 1: print("Ball passed robot")
          break

        # TODO: Check if this is actually linear
        # Change Neato velocity accordingly
        if -1 < x < 0:
          if sim_verbosity >= 3: print("Right")
          v_N = 0.2
        elif 0 < x < 1:
          if sim_verbosity >= 3: print("Left")
          v_N = -0.2
        else:
          if sim_verbosity >= 3: print("Stop")
          v_N = 0.0
        
        # Package all of our state information
        new_data = np.array([x, y, v_B, v_N])

        if sim_verbosity >= 2: 
            print(x," | ", y, " | ", v_B, " | ", v_N)

        # Initialize data if not already initialized
        # Otherwise update the existing data
        if num_step == 0:
            data = new_data
        else:
            data = np.vstack((data, new_data))

    # Let us know if the ball never passed the robot
    if y > 0:
        if sim_verbosity >= 1: print("Ball never passed robot")

    # Let us know how many steps the simulation ran for
    if sim_verbosity >= 1: print("Simulation ran for ", num_step + 1, "steps")

    return data

# Generate Data Based on Simulations

In [ ]:
# Sweep over different starting x positions, y positions and ball velocities for simulated data

first_run = True

# x varies from -1.5 to 1.5 m
for init_x in np.linspace(-1.5, 1.5, 10):
    # y varies from 1.0 to 2.0 m
    for init_y in np.linspace(1.0, 2.0, 10):
        # v_B varies from 0.1 to 1.0
        for init_v_B in np.linspace(0.1, 1.0, 10):
          # Neato velocity in initial step is arbitrary
          new_data = runSimulation(init_x, init_y, init_v_B, 0.0, sim_verbosity=0)

          # Initialize data structure on first run
          if first_run:
              sim_data = new_data
              first_run = False
          
          # Concatenate data on following runs
          else:
              sim_data = np.vstack((sim_data, new_data))

In [ ]:
# Check how much data we have
print(sim_data.shape)

(38440, 4)


## Partition Data

In [ ]:
"""
Partition data into x_data and y_data

X_data contains x, y, v_B
y_data contains v_N
"""

# Organize data into X and y
X_data = sim_data[:, :3]
print("Shape of x data: ", X_data.shape)
y_data = sim_data[:,3:]
print("Shape of y data: ", y_data.shape)

# Split data into training and testing
X_train , X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.33, random_state = 42)
print("X_train: ", type(X_train), X_train.shape)
print("y_train: ", type(y_train), y_train.shape)
print("X_test:  ", type(X_test), X_test.shape)
print("y_test:  ", type(y_test), y_test.shape)

Shape of x data:  (38440, 3)
Shape of y data:  (38440, 1)
X_train:  <class 'numpy.ndarray'> (25754, 3)
y_train:  <class 'numpy.ndarray'> (25754, 1)
X_test:   <class 'numpy.ndarray'> (12686, 3)
y_test:   <class 'numpy.ndarray'> (12686, 1)


# Train Model

## Run the training

In [ ]:
# Instantiate the model and put it in training mode
net = StarterNN()
net.train()
net.verbose = 0

# Training Session Parameters
num_epochs = 10000
learning_rate = 0.001
# batch_size = 50

# Set up optimizer for gradient descent
lossFunction, optimizer = net.getLossFunctionAndOptimizer(learning_rate)

# Set up data for input into net
X_train_var = Variable(torch.tensor(X_train.astype(np.float32)))
X_test_var = Variable(torch.tensor(X_test.astype(np.float32)))
y_train_var = Variable(torch.tensor(y_train.astype(np.float32)))
y_test_var = Variable(torch.tensor(y_test.astype(np.float32)))

grad_magnitudes = []

# Go through all the epochs
for epoch in range(num_epochs):
  # Clear the gradient
  optimizer.zero_grad()

  # Forward pass input data into 
  y_pred = net(X_train_var)
  loss = lossFunction(y_pred, y_train_var)
  loss.backward()
  optimizer.step()    # Does the update

  for name, param in net.named_parameters():
    if name == 'fc1.weight':
      grad_magnitudes.append(np.abs(param.grad.numpy()).mean())

  if epoch % 1000 == 0:
    print("epoch", epoch)
    for name, param in net.named_parameters():
      print(name, "value", param.data, "gradient", param.grad)

  # print(getAccuracyQuick(net, X_test_var, y_test_var))
  # print(loss)

epoch 0
fc1.weight value tensor([[ 0.1858, -0.3416,  0.2812]]) gradient tensor([[ 0.5063, -0.9246, -0.2886]])
fc1.bias value tensor([-0.2774]) gradient tensor([-0.9158])
epoch 1000
fc1.weight value tensor([[-0.0336, -0.0193,  0.1435]]) gradient tensor([[-1.9011e-05, -2.0187e-02,  1.8171e-02]])
fc1.bias value tensor([-0.0408]) gradient tensor([-0.0079])
epoch 2000
fc1.weight value tensor([[-0.0335,  0.0067,  0.0151]]) gradient tensor([[-6.6687e-07,  7.6705e-04,  1.0607e-03]])
fc1.bias value tensor([-0.0116]) gradient tensor([-0.0020])
epoch 3000
fc1.weight value tensor([[-0.0335,  0.0011,  0.0022]]) gradient tensor([[-6.3106e-08,  1.6564e-04,  6.8437e-05]])
fc1.bias value tensor([-0.0013]) gradient tensor([-0.0002])
epoch 4000
fc1.weight value tensor([[-0.0335,  0.0002,  0.0010]]) gradient tensor([[-3.3734e-08,  3.9040e-06,  1.5879e-06]])
fc1.bias value tensor([-3.0293e-05]) gradient tensor([-5.3408e-06])
epoch 5000
fc1.weight value tensor([[-0.0335,  0.0002,  0.0010]]) gradient tensor(

In [ ]:
X_fake = Variable(torch.tensor(np.array([0.5, 2.0, 0.4]).astype(np.float32)))
V_neato = net(X_fake)
print(V_neato)

tensor([-0.0159], grad_fn=<AddBackward0>)


# Validate against a Linear Regression Model from Numpy

In [ ]:
# Run linear regression with 
np.linalg.lstsq(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


(array([[-0.03352841],
        [ 0.00021674],
        [ 0.00095321]]),
 array([283.76834144]),
 3,
 array([172.76302836, 162.07371041,  51.9319929 ]))

In [ ]:
# Add a bias term
X_train_bias = np.hstack((X_train, np.ones((X_train.shape[0],1)) ))
# print(X_train_bias.shape)

X_train_sad = X_train_bias[:,[1,2,3]]
print(X_train_sad.shape)

(25754, 3)


In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
np.var(y_train)*y_train.shape[0]

317.3165690766484

In [ ]:
print(y_train)
print(max(y_train))
print(min(y_train))

[[0. ]
 [0.2]
 [0. ]
 ...
 [0. ]
 [0. ]
 [0.2]]
[0.2]
[-0.2]


In [ ]:
"""
Possibly look into later

# Create samplers for dataloaders
sampler_train = SubsetRandomSampler(np.arrange(len(data_train), dtype=np.float64))
sample_test = SubsetRandomSampler(np.arrange(len(data_teset), dtype=np.float64))

# Create TensorDataLoaders for getting data out of TensorDatasets
loader_train = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
loader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

# Variables for storing metrics
grad_magnitudes = []
losshist_train_x = []
losshist_train_y = []
losshist_test_x = []
losshist_test_y = []
